In [1]:
import gzip
import shutil
import wget
import regex as re
import pandas as pd
import numpy as np
import json

In [3]:
# Download the file

#Defining the zip file URL
url = "https://www.clarin.si/repository/xmlui/bitstream/handle/11356/1523/MaCoCu-sl-en.tmx.gz"

# Downloading the file by sending the request to the URL
corpus_file = wget.download(url)
print('Downloading Completed')

In [4]:
# Unzip the file
with gzip.open('MaCoCu-sl-en.tmx.gz', 'rb') as f_in:
    with open('MaCoCu-sl-en.tmx', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [5]:
corpus = open('MaCoCu-sl-en.tmx', "r").read()

In [6]:
# Create and view a sample of the corpus
corpus_sample = open('MaCoCu-sl-en.tmx', "r").read(5000)
print(corpus_sample)

<?xml version="1.0"?>
<tmx version="1.4">
 <header
   adminlang="C"
   srclang="en"
   o-tmf="PlainText"
   creationtool="bitextor"
   creationtoolversion="8.2"
   datatype="PlainText"
   segtype="sentence"
   creationdate="20220420T120753"
   o-encoding="utf-8">
 </header>
 <body>
   <tu tuid="1" datatype="Text">
    <prop type="score-bicleaner-ai">0.950</prop>
    <prop type="biroamer-entities">No</prop>
    <prop type="translation-direction">sl-orig</prop>
    <prop type="type">1:1</prop>
    <tuv xml:lang="en">
     <prop type="source-document">http://www.damapapir.si/en/ecology</prop>
     <prop type="checksum-seg">c48f3fc0460c3d67</prop>
     <prop type="paragraph-id">p10s2</prop>
    <prop type="english-variant-document">B</prop>
    <prop type="english-variant-domain">MIX</prop>
     <seg>This significantly contributes to a clean and healthy environment.</seg>
    </tuv>
    <tuv xml:lang="sl">
     <prop type="source-document">http://www.damapapir.si/sl/ekologija</prop>
     <

In [7]:
# Prepare all the regexes
# Compile all tus
tu_re = re.compile('<tu tuid=".*?>\n(.*?)<\/tu>', re.DOTALL)
# Compile relevant information inside tus
bi_score_re = re.compile('<prop type="score-bicleaner-ai">(.*?)</prop>')
biroamer_re = re.compile('<prop type="biroamer-entities">(.*?)</prop>')
translation_dir_re = re.compile('<prop type="translation-direction">(.*?)</prop>')
en_source_re = re.compile('<tuv xml:lang="en">.*?<prop type="source-document">(.*?)</prop>', re.DOTALL)
en_par_id_re = re.compile('<tuv xml:lang="en">.*?<prop type="paragraph-id">(.*?)</prop', re.DOTALL)
en_par_re = re.compile('<tuv xml:lang="en">.*?<seg>(.*?)</seg>', re.DOTALL)
en_var_doc_re = re.compile('<prop type="english-variant-document">(.*?)</prop>')
en_var_dom_re = re.compile('<prop type="english-variant-domain">(.*?)</prop>')
sl_source_re = re.compile('<tuv xml:lang="sl">.*?<prop type="source-document">(.*?)</prop>', re.DOTALL)
sl_par_id_re = re.compile('<tuv xml:lang="sl">.*?<prop type="paragraph-id">(.*?)</prop', re.DOTALL)
sl_par_re = re.compile('<tuv xml:lang="sl">.*?<seg>(.*?)</seg>', re.DOTALL)


In [10]:
# Create a list of all tus from the sample corpus
tus_list_sample = tu_re.findall(corpus_sample)
len(tus_list_sample)

4

In [11]:
# View the tus_list
tus_list_sample[1]

'    <prop type="score-bicleaner-ai">0.929</prop>\n    <prop type="biroamer-entities">No</prop>\n    <prop type="translation-direction">en-orig</prop>\n    <prop type="type">1:1</prop>\n    <tuv xml:lang="en">\n     <prop type="source-document">https://www.mesimedical.com/mesi-mtablet/mesi-mtablet-tbi/</prop>\n     <prop type="checksum-seg">e3c65172f8025636</prop>\n     <prop type="paragraph-id">p38s0</prop>\n    <prop type="english-variant-document">UNK</prop>\n    <prop type="english-variant-domain">B</prop>\n     <seg>MESI mTABLET TBI is not only an advanced Toe-Brachial Index measuring device.</seg>\n    </tuv>\n    <tuv xml:lang="sl">\n     <prop type="source-document">https://www.mesimedical.com/sl/mesi-mtablet/mesi-mtablet-tbi/</prop>\n     <prop type="checksum-seg">6bd3c104f3168684</prop>\n     <prop type="paragraph-id">p31s0</prop>\n     <seg>MESI mTABLET TBI ni samo napredna naprava za merjenje indeksa palec-nadlaket.</seg>\n    </tuv>\n   '

In [13]:
# Check if regexes work
regexes =  [bi_score_re, biroamer_re, translation_dir_re, en_source_re, en_par_id_re, en_par_re, en_var_doc_re, en_var_dom_re, sl_source_re, sl_par_id_re, sl_par_re]

for rex in regexes:
    test_list = rex.findall(tus_list_sample[1])
    print(test_list)

['0.929']
['No']
['en-orig']
['https://www.mesimedical.com/mesi-mtablet/mesi-mtablet-tbi/']
['p38s0']
['MESI mTABLET TBI is not only an advanced Toe-Brachial Index measuring device.']
['UNK']
['B']
['https://www.mesimedical.com/sl/mesi-mtablet/mesi-mtablet-tbi/']
['p31s0']
['MESI mTABLET TBI ni samo napredna naprava za merjenje indeksa palec-nadlaket.']


In [14]:
# Create a list of all tus from the corpus
tus_list = tu_re.findall(corpus)
len(tus_list)

3176311

In [15]:
# Inspect an instance
tus_list[:1]

['    <prop type="score-bicleaner-ai">0.950</prop>\n    <prop type="biroamer-entities">No</prop>\n    <prop type="translation-direction">sl-orig</prop>\n    <prop type="type">1:1</prop>\n    <tuv xml:lang="en">\n     <prop type="source-document">http://www.damapapir.si/en/ecology</prop>\n     <prop type="checksum-seg">c48f3fc0460c3d67</prop>\n     <prop type="paragraph-id">p10s2</prop>\n    <prop type="english-variant-document">B</prop>\n    <prop type="english-variant-domain">MIX</prop>\n     <seg>This significantly contributes to a clean and healthy environment.</seg>\n    </tuv>\n    <tuv xml:lang="sl">\n     <prop type="source-document">http://www.damapapir.si/sl/ekologija</prop>\n     <prop type="checksum-seg">745f997615be9df9</prop>\n     <prop type="paragraph-id">p10s2</prop>\n     <seg>S tem bomo preprečili onesnaževanje narave in ohranjali zdravo okolje.</seg>\n    </tuv>\n   ']

In [17]:
# Create a list of dictionaries from the tus_list based on regexes
tus_content = []

for i in tus_list:
	# Find all relevant information based on regexes
	bi_score = bi_score_re.search(i).group(1)
	biroamer = biroamer_re.search(i).group(1)
	translation_dir = translation_dir_re.search(i).group(1)
	en_source = en_source_re.search(i).group(1)
	en_par_id = en_par_id_re.search(i).group(1)
	en_par = en_par_re.search(i).group(1)
	en_var_doc = en_var_doc_re.search(i).group(1)
	en_var_dom = en_var_dom_re.search(i).group(1)
	sl_source = sl_source_re.search(i).group(1)
	sl_par_id = sl_par_id_re.search(i).group(1)
	sl_par = sl_par_re.search(i).group(1)
	# Add information to the dictionary
	current_tu = {"score_bicleaner_ai": float(bi_score), "biroamer_entities": biroamer, "translation_direction": translation_dir, "en_source": en_source, "en_par_id": en_par_id, "en_par": en_par, "en_var_doc": en_var_doc, "en_var_dom": en_var_dom, "sl_source": sl_source, "sl_par_id": sl_par_id, "sl_par": sl_par}
	# Append the dictionary to the list
	tus_content.append(current_tu)

print(len(tus_content))

# Print some instances of the tus_content
tus_content[:2]

3176311


[{'score_bicleaner_ai': 0.95,
  'biroamer_entities': 'No',
  'translation_direction': 'sl-orig',
  'en_source': 'http://www.damapapir.si/en/ecology',
  'en_par_id': 'p10s2',
  'en_par': 'This significantly contributes to a clean and healthy environment.',
  'en_var_doc': 'B',
  'en_var_dom': 'MIX',
  'sl_source': 'http://www.damapapir.si/sl/ekologija',
  'sl_par_id': 'p10s2',
  'sl_par': 'S tem bomo preprečili onesnaževanje narave in ohranjali zdravo okolje.'},
 {'score_bicleaner_ai': 0.929,
  'biroamer_entities': 'No',
  'translation_direction': 'en-orig',
  'en_source': 'https://www.mesimedical.com/mesi-mtablet/mesi-mtablet-tbi/',
  'en_par_id': 'p38s0',
  'en_par': 'MESI mTABLET TBI is not only an advanced Toe-Brachial Index measuring device.',
  'en_var_doc': 'UNK',
  'en_var_dom': 'B',
  'sl_source': 'https://www.mesimedical.com/sl/mesi-mtablet/mesi-mtablet-tbi/',
  'sl_par_id': 'p31s0',
  'sl_par': 'MESI mTABLET TBI ni samo napredna naprava za merjenje indeksa palec-nadlaket.'}]

In [18]:
# Save json

with open("Macocu-sl-en.json", "w") as file:
	json.dump(tus_content,file, indent= "")